# Exhaust Gas Receiver Heat Transfer

a) Applying the First Law of Thermodynamics, 

$$\dot U = \dot m_\mathrm{in} h_\mathrm{in} - \dot m_\mathrm{out} h_\mathrm{out} - Q + \dot W$$

Here, the $\dot W = 0$ since the volume is constant.

From $U = m c_v T$ and $\dot m_\mathrm{in} = \dot m_\mathrm{out}$,

$$m c_v \dot T = \dot m_\mathrm{in} \left(h_\mathrm{in} - h_\mathrm{out}\right) - Q$$
$$\dot T = \frac{1}{m c_v} \left[\dot m_\mathrm{in} \left(h_\mathrm{in} - h_\mathrm{out}\right) - Q \right]$$

b) The following conditions are given:

In [12]:
T_amb = 30     #degC Ambient temperature 
T_in = 255     #degC Exhaust gas temperature at the inlet
T_out = 253    #degC Exhaust gas temperature at the outlet
h_in = 5.487e5 #J/kg Specific enthalpy of the gas at 255 degC 
h_out = 5.466e5 #J/kg Specific enthalpy of the gas at 253 degC
dm = 82.3 #kg/s       Exhaust gas flow
A = 85 #m2            Surface area of the cylinder wall
k_cyl = 54  #W/m/K   thermal conductivity of the cylinder wall
t_cyl = 0.02   #m     Cylinder wall thickness
h_air = 10     #W/m2/K  Heat transfer coefficent of the air 

Q = dm * (h_in - h_out)
print('Heat transfer to the environment is {:3.2f}kW'.format(Q/1000))

Heat transfer to the environment is 172.83kW


We use the Newton's cooling to calculate the overall heat transfer coefficient.

$$Q = U A \Delta T$$

In [13]:
U = Q / A / ( 0.5*(T_in + T_out) - T_amb)
print('Overall heat transfer coefficient of the reciever is {:3.2f}W/m2/K'.format(U))

Overall heat transfer coefficient of the reciever is 9.08W/m2/K


We use following law of adding thermal resistance to find the heat transfer coefficient of the exhaust gas to the cylinder.

$$R_\mathrm{Total} = R_\mathrm{exh} + R_\mathrm{wall} + R_\mathrm{air}$$
$$\frac{1}{U} = \frac{1}{h_\mathrm{exh}} + \frac{t_\mathrm{cyl}}{k} + \frac{1}{h_\mathrm{air}}$$
$$h_\mathrm{exh} = \frac{1}{\frac{1}{U} - \frac{t_\mathrm{cyl}}{k} - \frac{1}{h_\mathrm{air}}}$$

In [14]:
h_exh = 1 / (1 / U - t_cyl / k_cyl - 1 / h_air)
print('The heat transfer coefficient for the exhaust gas is {:3.2f}W/m2/K'.format(h_exh))

The heat transfer coefficient for the exhaust gas is 102.09W/m2/K


c) With the insulation, the thermal conductivity will change greatly. Here, the thermal resistance of the cylinder wall is ignored. We assume that the heat transfer coefficients for air and exhaust gas don't change since their flow has not changed. Also, we use the outlet temperature unchanged from the case without insulation for the first calculation.

In [18]:
t = 0.1       #m       Thickness of the rock wool insulation
k_rw = 0.045  #W/m2/K  Heat transfer coefficient of rook wool insulation
U = 1 / ( 1 / h_exh + t / k_rw + 1 / h_air)

Q = U * A * (0.5 * (T_in + T_out) - T_amb)

print('The overall heat transfer coefficient is {:3.2f}W/m2/K.'.format(U))
print('Initial calculation shows that the heat transfer rate is {:3.2f}kW.'.format(Q/1000))

The overall heat transfer coefficient is 0.43W/m2/K.
Initial calculation shows that the heat transfer rate is 8.16kW.


# Exhaust Gas Recirculation Cooler Design

1) The following is given for the operational conditions and requirement at 100% load

In [190]:
from iapws import IAPWS97
from iapws import HumidAir
import numpy as np

# Operating conditions and enthalpy of the exhaust gas
T_egr_in = 440       #K
T_egr_out = 350     #K   
h_egr_in = 4.55e+05 #J/kg
h_egr_out = 3.58e+05 #J/kg
dm_exh = 82.3        #kg/s
r_egr = 0.4          # EGR ratio
dm_egr = r_egr * dm_exh       
print('Exhaust gas flow for EGR is {:3.2f}kg/s'.format(dm_egr))

# Operating conditions and properties of water at average temperature
T_w_in = 310
T_w_max = 330
waterP = IAPWS97(P = 0.3, T = 0.5 * (T_w_in + T_w_max), x = 0) 
    # assuming 3 barA as operating pressure 
print('Water properties')
print('  - heat capacity (Cp): {:3.2f}kJ/kg/K'.format(waterP.cp * 1000))

Exhaust gas flow for EGR is 32.92kg/s
Water properties
  - heat capacity (Cp): 4178.53kJ/kg/K


Now we can calculate the required heat transfer rate and the flow of water

In [191]:
# Required heat transfer from exhaust gas to water
Q = dm_egr * (h_egr_in - h_egr_out)
print('{:3.1f}kW of heat transfer is required'.format(Q/1000))

# Required water flow 
dm_w = Q / (waterP.cp * 1000 * (T_w_max - T_w_in))
print('{:.2f}kg/s of water flow is required.'.format(dm_w))

3193.2kW of heat transfer is required
38.21kg/s of water flow is required.


2) The following dimensions and thermal properties are given for the design

In [201]:
t = 0.005 #the gap of the channel in meter
w = 0.5  #width of the channel in meter
N = 100   #Number of channels for each side

nu_exh = 5.20e-6 #kinematic viscosity of air
nu_water = waterP.nu

k_exh = 0.033 # Thermal conductivity in W/m/K
k_water = waterP.k 

rho_exh = 4.40 # Air density in kg/m3
rho_water = waterP.rho # Water density in kg/m3

mu_exh = nu_exh * rho_exh
mu_water = nu_water * rho_water

cp_exh = 1073.7 #specific heat capacity at constant pressure in  J/kg/K
cp_water = waterP.cp * 1000

print('Water properties')
print('  - Dynamic visicosity: {:.2e}m2/s'.format(mu_water))
print('  - Thermal conductivity: {:.3f}W/m/K'.format(k_water))
print('  - Density of water: {:3.1f}kg/m3'.format(rho_water))
print('  - Specific heat capacity: {:4.0f}J/kg/K'.format(cp_water))

Water properties
  - Dynamic visicosity: 5.77e-04m2/s
  - Thermal conductivity: 0.637W/m/K
  - Density of water: 989.5kg/m3
  - Specific heat capacity: 4179J/kg/K


Reynold's number for each channel is calculated first.
$$Re_D = \frac{V D_h}{\nu}$$

In [193]:
#Hydraulic diameter of a channel
Ac = t * w
P = 2 * w
Dh = 4 * Ac / P

# Velocity of the flow
V_water = dm_w / rho_water / (N * Ac)
V_exh = dm_egr / rho_exh / (N * Ac)
print('Velocity of exhaust gas and water are {:.2f}m/s and {:.2f}m/s'.format(V_exh, V_water))

#Reynold's number
Re_water = V_water * Dh / nu_water
Re_exh = V_exh * Dh / nu_exh
print('Reynold''s number for exhaust gas and water: {:.2f}, {:.2f}'.format(Re_exh, Re_water) )

Velocity of exhaust gas and water are 29.93m/s and 0.15m/s
Reynolds number for exhaust gas and water: 57552.45, 2649.95


The flows both in cooling and heating sides are turbulent. Therefore, we may use the correlation for a circular tube that is given as:

$$Nu_D = 0.023Re_D^{4/5}Pr^n$$

where, $n = 0.4$ for heating and $n = 0.3$ for cooling. 

Then, the heat transfer coefficients for water and exhaust gas are calculated as follows:

In [194]:
# Prandtl number
Pr_water = cp_water * mu_water / k_water
Pr_exh = cp_exh * mu_exh / k_exh
print('Prandtl number for:')
print('  - Water: {:3.2f}'.format(Pr_water))
print('  - Exhaust gas: {:3.2f}'.format(Pr_exh))
print('')

# Nusselt number
Nu_water = 0.023 * Re_water**0.8 * Pr_water**0.3
Nu_exh = 0.023 * Re_exh**0.8 * Pr_exh**0.4
print('Nusselt number for:')
print('  - Water: {:3.2f}'.format(Nu_water))
print('  - Exhaust gas: {:3.2f}'.format(Nu_exh))
print('')

# Heat transfer coeffient
h_water = Nu_water * k_water / Dh
h_exh = Nu_exh * k_exh / Dh
print('Heat transfer coefficient in J/m2/K for:')
print('  - Water side: {:3.2f}'.format(h_water))
print('  - Exhaust gas side: {:3.2f}'.format(h_exh))
print('')

# Overall heat transfer coefficient
U = 1 / (1/h_water + 1/h_exh)
print('Overall heat transfer coefficient is {:3.2f}W/m2/K'.format(U))

Prandtl number for:
  - Water: 3.78
  - Exhaust gas: 0.74

Nusselt number for:
  - Water: 18.78
  - Exhaust gas: 131.37

Heat transfer coefficient in J/m2/K for:
  - Water side: 1196.40
  - Exhaust gas side: 433.53

Overall heat transfer coefficient is 318.22W/m2/K


3) We will first find the logarithmic mean temperature difference. Then we use heat transfer equation to find the required heat exchange area.

$$\Delta T_\rm{lm} = \frac{\left(T_\rm{EGR,in} - T_\rm{w,max}\right) - \left(T_\rm{EGR,out} - T_\rm{w,in}\right)}{\ln\left(\frac{T_\rm{EGR,in} - T_\rm{w,max}}{T_\rm{EGR,out} - T_\rm{w,in}}\right)}$$

$$A = \frac{Q}{U\Delta T_\rm{lm}}$$

The length of the each channel is then determined as follows:

$$L = \frac{A}{N w}$$

In [195]:
# Logarithmic mean temperature difference
delta_T_A = T_egr_in - T_w_max
delta_T_B = T_egr_out - T_w_in
delta_T_lm = (delta_T_A - delta_T_B) / np.log(delta_T_A / delta_T_B) 
print('Logarighmic mean temperature difference is {:3.0f}K'.format(delta_T_lm))

# Required area of heat exchage
A = Q / (U * delta_T_lm)
print('Required area of heat exchange is {:.2f}m2'.format(A))

# Length of the channel
L = A / 2 * (N - 1) / w
print('The length of the channel is {:.2f}m'.format(L))

Logarighmic mean temperature difference is  69K
Required area of heat exchange is 145.02m2
The length of the channel is 2.90m


3) Use effectivity-NTU method to calculate the new conditions. This is a counter current heat exchanger. Therefore, following equations should be used.

$$\epsilon = \frac{1 - \exp\left[-\rm{NTU}\left(1 - C_\rm{min} / C_\rm{max}\right)\right]}{1 - C_\rm{min} / C_\rm{max} \exp\left[-NTU\left(1 
- C_\rm{min} / C_\rm{max}\right)\right]}$$

Following conditions and properties are given.


In [196]:
T_egr_in = 420       #K
cp_egr_in = 1079    #J/kg/K
dm_exh = 42.7        #kg/s
r_egr = 0.4          # EGR ratio
dm_egr = r_egr * dm_exh       
rho_exh = 2.48       # kg/m3\
nu_exh = 9.64e-6     # m2/s
k_exh = 0.0346       # W/m/K
print('Exhaust gas flow for EGR is {:3.2f}kg/s'.format(dm_egr))


Exhaust gas flow for EGR is 17.08kg/s


We will assume that the heat transfer coefficient at the water side will be constant. For the exhaust gas side, it should re-calculated.

In [197]:
# Velocity of the flow
V_exh = dm_egr / rho_exh / (N * Ac)
print('Velocity of exhaust gas is {:.2f}m/s'.format(V_exh))

#Reynold's number
Re_exh = V_exh * Dh / nu_exh
print('Reynold''s number for exhaust gas: {:.2f}'.format(Re_exh) )

# Prandtl number
Pr_exh = cp_exh * mu_exh / k_exh
print('Prandtl number for:')
print('  - Exhaust gas: {:3.2f}'.format(Pr_exh))
print('')

# Nusselt number
Nu_exh = 0.023 * Re_exh**0.8 * Pr_exh**0.4
print('Nusselt number for:')
print('  - Exhaust gas: {:3.2f}'.format(Nu_exh))
print('')

# Heat transfer coeffient
h_exh = Nu_exh * k_exh / Dh
print('Heat transfer coefficient in J/m2/K for:')
print('  - Exhaust gas side: {:3.2f}'.format(h_exh))
print('')

# Overall heat transfer coefficient
U = 1 / (1/h_water + 1/h_exh)
print('Overall heat transfer coefficient is {:3.2f}W/m2/K'.format(U))

Velocity of exhaust gas is 27.55m/s
Reynolds number for exhaust gas: 28577.17
Prandtl number for:
  - Exhaust gas: 0.71

Nusselt number for:
  - Exhaust gas: 73.63

Heat transfer coefficient in J/m2/K for:
  - Exhaust gas side: 254.76

Overall heat transfer coefficient is 210.03W/m2/K


Calculate heat capacity of each side first.

$$C = \dot m c_p $$

In [198]:
C_exh = dm_egr * cp_egr_in
C_w = dm_w * cp_water
print('The heat capacity in J/K/s for:')
print('  - Water: {:3.2f}'.format(C_w))
print('  - Exhaust gas: {:3.2f}'.format(C_exh))

The heat capacity in J/K/s for:
  - Water: 159662.00
  - Exhaust gas: 18429.32


Therefore, $C_\rm{min} = C_\rm{exh} = 18430$ J/K/s. Then NTU can be calculated as 

$$NTU = \frac{UA}{C}$$

In [199]:
NTU = U * A / C_exh
print('NTU is {:3.2f}'.format(NTU))

NTU is 1.65


Now calculate $\epsilon$ using $\epsilon$-NTU method.

$$epsilon = \frac{C_c\left(T_{c,o} - T_{c,i}\right)}{C_\rm{min}\left(T_{h,i} - T_{c,i}\right)} = \frac{C_h\left(T_{h,i} - T_{h,o}\right)}{C_\rm{min}\left(T_{h,i} - T_{c,i}\right)}$$

In [200]:
Cr = C_exh / C_w
epsilon = (1 - np.exp(-NTU*(1 - Cr )))/(1 - Cr * np.exp(-NTU*(1 - Cr)))
print('Effectivity is {:3.2}'.format(epsilon))
T_w_out = epsilon * Cr * (T_egr_in - T_w_in) + T_w_in
T_egr_out = T_egr_in - epsilon * (T_egr_in - T_w_in) 
print('Outlet temperature:')
print('  - water: {:3.2f}K'.format(T_w_out))
print('  - exhaust gas: {:3.2f}K'.format(T_egr_out))


Effectivity is 0.79
Outlet temperature:
  - water: 320.02K
  - exhaust gas: 333.17K
